# Generate data for cache db
This notebook can be used to generate data for Recommendation Service Cache DB

In [1]:
import os

import numpy as np
import pandas as pd

## Read data

In [2]:
data_dir = 'data_0419_0'

In [3]:
db_data_dir = os.path.join(data_dir, 'data4db')
if not os.path.exists(db_data_dir):
    os.makedirs(db_data_dir)

### product_info table

```SQL
CREATE TABLE product_info (
    product_id INT,
    name VARCHAR(256),
    category VARCHAR(50),
    sizes VARCHAR(50),
    vendor VARCHAR(50),
    description VARCHAR(256),
    buy_price REAL,
    department VARCHAR(10),
    PRIMARY KEY (product_id)
);

COPY product_info FROM '<<DATA_PATH>>/products.csv' DELIMITER ',' CSV HEADER;
```

In [4]:
products = pd.read_csv(os.path.join(data_dir, 'products.csv'))
products.sample()

,id,name,category,sizes,vendor,description,buy_price,department
1173,1174,TETE - Bright lilac Suspenders for Boys,Suspenders,XS-XXL,Adidas,TETE - Bright lilac Suspenders for Boys by Adi...,16.45,Boys


In [5]:
products.rename(columns={'id': 'product_id'}, inplace=True)

In [6]:
products.to_csv(os.path.join(db_data_dir, 'products.csv'), index=False)

### coupon_info table

```SQL
CREATE TABLE coupon_info (
  coupon_id INT,
  coupon_type REAL,
  department VARCHAR(10),
  discount INT,
  how_many_products_required INT,
  product_mean_price REAL,
  products_available INT,
  start_date VARCHAR(10),
  end_date VARCHAR(10),
  PRIMARY KEY (coupon_id)
);

COPY coupon_info FROM '<<DATA_PATH>>/coupon_info.csv' DELIMITER ',' CSV HEADER;
```

In [7]:
coupons = pd.read_csv(os.path.join(data_dir, 'coupons.csv'))
coupons.sample()

,id,type,department,discount,how_many,start_date,end_date
521,522,just_discount,Sport,8,1,2011-08-30,2011-09-27


In [8]:
coupons.rename(columns={'id': 'coupon_id', 'type': 'coupon_type', 'how_many': 'how_many_products_required'}, inplace=True)
coupons.start_date = pd.to_datetime(coupons.start_date, format='%Y-%m-%d')
coupons.end_date = pd.to_datetime(coupons.end_date, format='%Y-%m-%d')

In [9]:
coupon_product = pd.read_csv(os.path.join(data_dir, 'coupon_product.csv'))
coupon_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1700 entries, 0 to 1699
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   coupon_id   1700 non-null   int64
 1   product_id  1700 non-null   int64
dtypes: int64(2)
memory usage: 26.7 KB


In [10]:
# In 'department'-type coupons, change 'how_many_products_required' from -1 to 1
coupons.loc[coupons.coupon_type == 'department', 'how_many_products_required'] = 1

# coupon_product table is missing mapping for 'department'-type coupons, fill that in
dept_coupon_prod = pd.merge(
    coupons.loc[coupons.coupon_type == 'department'][['coupon_id', 'department']],
    products[['product_id', 'department']],
    on='department',
    how='left').drop('department', axis=1)

coupon_product = coupon_product.append(dept_coupon_prod)
coupon_product

,coupon_id,product_id
0,1,128
1,1,280
2,1,396
3,1,2081
4,2,2883
...,...,...
15810,962,2986
15811,962,2987
15812,962,2990
15813,962,2991


In [11]:
coupon_product_price = pd.merge(coupon_product, coupons[['coupon_id']], on='coupon_id', how='left')\
    .merge(products[['product_id', 'buy_price']], on='product_id', how='left').drop_duplicates()
coupon_product_price

,coupon_id,product_id,buy_price
0,1,128,3.49
1,1,280,3.04
2,1,396,5.88
3,1,2081,16.23
4,2,2883,9.12
...,...,...,...
17510,962,2986,15.93
17511,962,2987,2.57
17512,962,2990,8.58
17513,962,2991,8.23


In [12]:
coupon_stats = pd.pivot_table(coupon_product_price, values=['product_id', 'buy_price'], index='coupon_id',
                              aggfunc={
                                  'product_id': len,
                                  'buy_price': np.mean
                              })
coupon_stats.buy_price = coupon_stats.buy_price.round(2)
coupon_stats.rename(columns={'buy_price': 'product_mean_price', 'product_id': 'products_available'}, inplace=True)

In [13]:
coupon_info = pd.merge(coupons, coupon_stats, on='coupon_id', how='left')
coupon_info

,coupon_id,coupon_type,department,discount,how_many_products_required,start_date,end_date,product_mean_price,products_available
0,1,buy_all,Men,10,4,2010-01-01,2010-01-08,7.16,4
1,2,buy_more,Men,23,3,2010-01-01,2010-01-15,9.12,1
2,3,just_discount,Men,12,1,2010-01-01,2010-01-25,1.13,1
3,4,buy_all,Sport,49,4,2010-01-01,2010-01-25,5.85,4
4,5,buy_more,Sport,20,4,2010-01-01,2010-01-11,9.59,1
...,...,...,...,...,...,...,...,...,...
966,967,buy_more,Girls,33,2,2012-12-21,2013-01-13,4.81,1
967,968,just_discount,Men,10,1,2012-12-22,2013-01-18,1.05,1
968,969,buy_more,Women,27,5,2012-12-27,2013-01-14,6.41,1
969,970,buy_all,Boys,28,3,2012-12-29,2013-01-01,11.36,3


In [14]:
coupon_info.to_csv(os.path.join(db_data_dir, 'coupon_info.csv'), index=False)

### coupon_product table

```SQL
CREATE TABLE coupon_product (
    coupon_id INT,
    product_id INT,
    FOREIGN KEY (coupon_id) REFERENCES coupon_info(coupon_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
);

COPY coupon_product FROM '<<DATA_PATH>>/coupon_product.csv' DELIMITER ',' CSV HEADER;
```

In [15]:
coupon_product.drop_duplicates().to_csv(os.path.join(db_data_dir, 'coupon_product.csv'), index=False)

### customer_info table

```SQL
CREATE TABLE customer_info (
  customer_id INT,
  gender VARCHAR(1),
  age INT,
  mean_buy_price REAL,
  unique_products_bought INT,
  unique_products_bought_with_coupons: INT,
  total_items_bought: INT,
  mean_discount_received: REAL,
  total_coupons_used: INT
  PRIMARY KEY (customer_id)
);

COPY customer_info FROM '<<DATA_PATH>>/customer_info.csv' DELIMITER ',' CSV HEADER;
```

In [16]:
customers = pd.read_csv(os.path.join(data_dir, 'customers.csv'))
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           1000 non-null   int64 
 1   name         1000 non-null   object
 2   gender       1000 non-null   object
 3   age          1000 non-null   int64 
 4   phone        1000 non-null   object
 5   address      1000 non-null   object
 6   city         1000 non-null   object
 7   state        1000 non-null   object
 8   postalCode   1000 non-null   int64 
 9   country      1000 non-null   object
 10  creditLimit  1000 non-null   int64 
dtypes: int64(4), object(7)
memory usage: 86.1+ KB


In [17]:
customers = customers[['id', 'gender', 'age']].rename(columns={'id': 'customer_id'})
customers

,customer_id,gender,age
0,1,M,79
1,2,F,38
2,3,F,84
3,4,F,24
4,5,M,29
...,...,...,...
995,996,F,86
996,997,M,36
997,998,M,80
998,999,F,48


In [18]:
orders = pd.read_csv(os.path.join(data_dir, 'orders.csv'))
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302793 entries, 0 to 302792
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   id           302793 non-null  int64 
 1   customer_id  302793 non-null  int64 
 2   order_date   302793 non-null  object
dtypes: int64(2), object(1)
memory usage: 6.9+ MB


In [19]:
orders.rename(columns={'id': 'order_id'}, inplace=True)

In [20]:
order_details = pd.read_csv(os.path.join(data_dir, 'order_details.csv'))
order_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2162820 entries, 0 to 2162819
Data columns (total 7 columns):
 #   Column            Dtype  
---  ------            -----  
 0   id                int64  
 1   order_id          int64  
 2   product_id        int64  
 3   quantity_ordered  int64  
 4   original_price    float64
 5   buy_price         float64
 6   coupon_id         float64
dtypes: float64(3), int64(4)
memory usage: 115.5 MB


In [21]:
order_details.drop('id', axis=1, inplace=True)

In [22]:
cust_orders = pd.merge(orders, order_details, on='order_id', how='left')
cust_orders['discount'] = 100 * ((cust_orders.original_price - cust_orders.buy_price) / cust_orders.original_price)

In [23]:
cust_stats = pd.pivot_table(cust_orders,
                            values=['product_id', 'buy_price', 'coupon_id', 'discount'],
                            index='customer_id',
                            aggfunc={
                                'product_id': lambda x: len(set(x)),  # sum of unique products bought
                                'coupon_id': lambda x: x.notnull().sum(),  # total coupons used
                                'discount': lambda x: np.round(np.mean(x), decimals=2),  # mean discount used
                                'buy_price': lambda x: np.round(np.mean(x), decimals=2)  # mean price paid
                            })
cust_stats.rename(columns={
    'product_id': 'unique_products_bought',
    'coupon_id': 'total_coupons_used',
    'discount': 'mean_discount_received',
    'buy_price': 'mean_buy_price'
}, inplace=True)

cust_stats['unique_products_bought_with_coupons'] = cust_orders.loc[cust_orders.coupon_id.notnull()]\
    .groupby('customer_id').agg({'product_id': 'nunique'})
cust_stats.fillna(value=0, inplace=True)

cust_stats['total_items_bought'] = cust_orders.groupby('customer_id').count().product_id

In [24]:
customer_info = pd.merge(customers[['customer_id', 'gender', 'age']], cust_stats, on='customer_id', how='left')
customer_info

,customer_id,gender,age,mean_buy_price,total_coupons_used,mean_discount_received,unique_products_bought,unique_products_bought_with_coupons,total_items_bought
0,1,M,79,11.62,285.0,9.16,866.0,232.0,1102.0
1,2,F,38,14.29,984.0,11.10,1566.0,634.0,2980.0
2,3,F,84,11.43,209.0,10.95,510.0,183.0,629.0
3,4,F,24,10.65,154.0,10.13,501.0,148.0,564.0
4,5,M,29,5.68,0.0,0.00,2.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...
995,996,F,86,12.58,2826.0,10.69,2310.0,1092.0,9264.0
996,997,M,36,12.70,932.0,9.67,1709.0,603.0,3365.0
997,998,M,80,13.38,118.0,11.11,327.0,107.0,355.0
998,999,F,48,13.28,102.0,6.75,465.0,99.0,527.0


In [26]:
customer_info.dropna(inplace=True)
customer_info.total_coupons_used = customer_info.total_coupons_used.astype(int)
customer_info.unique_products_bought = customer_info.unique_products_bought.astype(int)
customer_info.unique_products_bought_with_coupons = customer_info.unique_products_bought_with_coupons.astype(int)
customer_info.total_items_bought = customer_info.total_items_bought.astype(int)

In [27]:
customer_info.to_csv(os.path.join(db_data_dir, 'customer_info.csv'), index=False)